#### Linear system of equations

Linear system of equations solves $b=Ax$ for $x$, $A\in \mathbf{R}^{m \times m}$, full rank, we can use either Cholesky or LU with partial pivoting

#### Cholesky factorization

Similar to least squares problem, we can transform the problem using Cholesky decomposition

$$Ax=b \Longleftrightarrow A^TAx=A^Tb \Longleftrightarrow Bx=c \Longleftrightarrow LL^Tx=c$$

Then

* solve $Ly=c$ using `forward` substitution
* solve $L^Tx=y$ using `backward` substitution

In [9]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [10]:
def cholesky_factorization(A):
    m = A.shape[0]
    l_mat = A.copy().astype(float)

    for k in range(m):
        if l_mat[k, k] <= 0:
            return print('Input is not positive definite')

        # Follow the first step, iteratively apply to a smaller and smaller K
        l_mat[k+1:, k+1:] -= np.outer(l_mat[k+1:, k], l_mat[k+1:, k]) / l_mat[k, k]
        l_mat[k:, k] /= np.sqrt(l_mat[k, k])

    return np.tril(l_mat)

def forward_substitution(L, b):
    m, n = L.shape
    x = np.zeros(n)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

def back_substitution(R, b):
    m, n = R.shape
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(R[i, i + 1:], x[i + 1:])) / R[i, i]
    return x

In [11]:
np.random.seed(42)
m = 50

A = np.random.randn(m, m)
x = np.random.randn(m)
b = A @ x

In [12]:
L = cholesky_factorization(A.T @ A)
c_ch = A.T @ b
y_ch = forward_substitution(L, c_ch)
x_ch = back_substitution(L.T, y_ch)
print(np.linalg.norm(x_ch - x))

1.4616675373045904e-13


#### LU with partial pivoting

We first find $P$, $L$, $U$ such that $PA=LU$

Then, we rewrite $Ax=b$ as $LUx=Pb$ and

* solve $Ly=Pb$ using `forward` substitution
* solve $Ux=y$ using `backward` substitution

In [13]:
def lu_factorization(A):
    m, n = A.shape
    u_mat = A.copy().astype(float)
    l_mat = np.identity(m)
    p_mat = np.identity(m)

    for k in range(m-1):
        # Find pivot
        pivot = np.argmax(np.abs(u_mat[k:, k])) + k

        if pivot != k:
            # Swap rows in u, p, and l
            u_mat[[k, pivot], :] = u_mat[[pivot, k], :]
            p_mat[[k, pivot], :] = p_mat[[pivot, k], :]
            l_mat[[k, pivot], :k] = l_mat[[pivot, k], :k]

        for j in range(k + 1, m):
            l_mat[j, k] = u_mat[j, k] / u_mat[k, k]
            # Subtract multiply of kth row from jth row
            u_mat[j, k:] -= l_mat[j, k] * u_mat[k, k:]

    return p_mat, l_mat, u_mat

In [14]:
p, l, u = lu_factorization(A)
y_lu = forward_substitution(l, p @ b)
x_lu = back_substitution(u, y_lu)

print(np.linalg.norm(x_lu - x))

2.5865046977186647e-14
